# Clustering of diffrent nodes 

We used the power of Neo4J do cluster a selected range of diffrent features in the dataset

In [13]:
#Imports
from neo4j import GraphDatabase, basic_auth
from py2neo import Graph
import neo4jupyter
import pandas as pd
from vis_class import vis_class

In [21]:
#Create db connector
login = open("Login.txt", "r")
uri = login.readline()
user = login.readline()
password = login.readline()
login.close()

driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
session = driver.session(database="neo4j")
visualizer = vis_class()

<IPython.core.display.Javascript object>

Vis class was created


## Starting clustering

We start simple and cluster each character each movie they played in. It the name and a list with their movies will be returned

In [22]:
cluster_each = """MATCH (p:Person)-[:APPEARS_IN]->(m:Movie) 
                Return DISTINCT p.name AS `Name character`, collect(m.name) As Movies"""
df = pd.DataFrame (session.read_transaction(
                    lambda tx: tx.run(cluster_each).data()), columns = ['Name character','Movies'])
df.head()

AuthError: {code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.}

In [16]:
cluster_each = """MATCH (p:Person)-[a:APPEARS_IN]->(m:Movie) 
                Return DISTINCT p AS source_node, 
                                id(p) AS source_id,
                                a,
                                m AS target_node,
                                id(m) As target_id 
                LIMIT 20"""

In [17]:
vis_class.drawGraph(session, {'Person':'name', 'Movie':'name'}, cluster_each)

AuthError: {code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.}

Here we get the number of characters in each movie and cluster them if they have more then 20 characters

In [ ]:
cluster_by_count = """MATCH pattern=((m:Movie)-[a:APPEARS_IN]-(p:Person)) 
                    with count(pattern) as connections, m
                    Where connections > 20
                    Return m.name AS Movie, connections As Characters"""
df = pd.DataFrame (session.read_transaction(
                            lambda tx: tx.run(cluster_by_count).data()), columns = ['Movie','Characters'])

df.head()

We group the data in three diffrent groups depending on the count of characters

In [ ]:
#With cases 
cluster_by_count_categories = """MATCH pattern=((m:Movie)-[a:APPEARS_IN]-(p:Person)) 
                                with count(pattern=((m:Movie)-[a:APPEARS_IN]-(p:Person))) as connections, m
                                Return 
                                CASE
                                  WHEN connections <= 20 THEN "Group 1: " + m.name
                                  WHEN connections <= 30      THEN "Groupe 2: " + m.name
                                  ELSE "Group 3: " + m.name
                                END AS Clusters
                                Order by connections"""
df = pd.DataFrame (session.read_transaction(
                            lambda tx: tx.run(cluster_by_count_categories).data()), columns = ['Clusters'])

df.head()

We cluster characters which are in in two diffrent movies

In [ ]:
cluster_by_characters_interact_in_diffrent_movies = """MATCH (p1:Person)-[a1:APPEARS_IN]-(m1:Movie)-[a2:APPEARS_IN]-(p2:Person)-[a3:APPEARS_IN]-(m2:Movie)
                                WHERE m1.name <> m2.name
                                RETURN p2.name as Characters, p1.name as CharacterConnected, m1.name as ViaMovie"""
df = pd.DataFrame (session.read_transaction(
                            lambda tx: tx.run(cluster_by_characters_interact_in_diffrent_movies).data()), columns = ['Characters', 'CharacterConnected', 'ViaMovie'])

df.head()

In [ ]:
cluster_by_characters_interact_in_diffrent_movies = """MATCH (p1:Person)-[a1:APPEARS_IN]-(m1:Movie)-[a2:APPEARS_IN]-(p2:Person)-[a3:APPEARS_IN]-(m2:Movie)
                                WHERE m1.name <> m2.name
                                RETURN p1 AS source_node, 
                                id(p1) AS source_id,
                                a1,
                                p2 AS target_node,
                                id(p2) As target_id 
                                LIMIT 20"""

In [ ]:
vis_class.drawGraph(session, {'Person':'name', 'Person':'name'}, cluster_each)

![title](img/ConnectedWithOthers.png)

## Close the connection

In [ ]:
#Close all connections
driver.close()
session.close()
drive = None
session = None